# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
import os
os.chdir('d:/Users/mokky/Documents/GitHub/nus-iss/PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

d:\Users\mokky\Documents\GitHub\nus-iss\PRS-PM-ISY5002-GROUP5\SystemCode


In [4]:
from datapipeline import Datapipeline
dpl = Datapipeline()

#### Read data from file

In [5]:
file_dir = './data/uc1/'
raw_data_path = './Data Exploration/data/Latest_ParkwaySampleDataForProject_12_WithLabelNoFormula.xlsx'
proc_data_path = dpl.transform_raw_data(raw_data_path, split_hosp=False)
print(proc_data_path)

./Data Exploration/data/all_hosp_data.pkl


In [6]:
X_train_file_paths, y_train_file_path, X_test_file_paths, y_test_file_path = dpl.transform_train_test_data(proc_data_path)
print(X_train_file_paths, y_train_file_path, X_test_file_paths, y_test_file_path)

Processing train X rows: 0 ~ 10000
Processing train X rows: 10000 ~ 20000
Processing train X rows: 20000 ~ 30000
Processing train X rows: 30000 ~ 40000
Processing train X rows: 40000 ~ 50000
Processing train X rows: 50000 ~ 60000
Processing train X rows: 60000 ~ 70000
Processing train X rows: 70000 ~ 80000
Processing train X rows: 80000 ~ 90000
Processing train X rows: 90000 ~ 100000
Processing train X rows: 100000 ~ 110000
Processing train X rows: 110000 ~ 120000
Processing train X rows: 120000 ~ 130000
Processing train X rows: 130000 ~ 140000
Processing train X rows: 140000 ~ 150000
Processing train X rows: 150000 ~ 160000
Processing train X rows: 160000 ~ 170000
Processing train X rows: 170000 ~ 180000
Processing train X rows: 180000 ~ 190000
Processing train X rows: 190000 ~ 200000
Processing train X rows: 200000 ~ 210000
Processing train X rows: 210000 ~ 220000
Processing train X rows: 220000 ~ 230000
Processing train X rows: 230000 ~ 240000
Processing train X rows: 240000 ~ 25000

In [7]:
df_X_train = pd.concat([pd.read_pickle(file_path)
                        for file_path in X_train_file_paths])
print(f'X_train {df_X_train.shape}')
    
df_y_train = pd.read_pickle(y_train_file_path)
print(f'y_train {df_y_train.shape}')

df_X_test = pd.concat([pd.read_pickle(file_path)
                       for file_path in X_test_file_paths])
print(f'X_test {df_X_test.shape}')

df_y_test = pd.read_pickle(y_test_file_path)
print(f'y_test {df_y_test.shape}')

MemoryError: Unable to allocate 82.8 GiB for an array with shape (38885, 285908) and data type float64

## Get feature Importances

In [ ]:
from sklearn.ensemble import RandomForestRegressor

impt_threshold = 0.0

save_folder_paths = os.path.dirname(X_train_file_paths[0])
    
# forest
forest = RandomForestRegressor(random_state=42)
forest.fit(df_X_train, df_y_train)
    
forest_pkl_file_path = f'{save_folder_paths}/all_hosp_forest_model_uc1.pkl'
with open(forest_pkl_file_path, 'wb') as file:
    pickle.dump(forest, file)

# feature importance
print(f'{sum(forest.feature_importances_ > impt_threshold)} / {len(forest.feature_importances_)}')
feature_importance_file_path = f'{save_folder_paths}/all_hosp_forest_feat_impt_uc1.npy'
np.save(feature_importance_file_path, forest.feature_importances_ > impt_threshold)

# new training files
df_X_train_new = df_X_train.loc[:, forest.feature_importances_ > impt_threshold]
print(df_X_train_new.shape)
new_X_train_pkl_file_path = f'{save_dir}/all_hosp_new_train_X_uc1.pkl'
df_X_train_new.to_pickle(new_X_train_pkl_file_path)

# new test files
df_X_test_new = df_X_test.loc[:, forest.feature_importances_ > impt_threshold]
print(df_X_test_new.shape)
new_X_test_pkl_file_path = f'{save_dir}/all_hosp_new_test_X_uc1.pkl'
df_X_test_new.to_pickle(new_X_test_pkl_file_path)

print('forest:\n', forest_pkl_file_path)
print('feature importance:\n', feature_importance_file_path)
print('New X_train:\n', new_X_train_pkl_file_path)
print('New X_test:\n', new_X_test_pkl_file_path)

## Classify y

In [ ]:
def convert_y(df):
    #df1 = dpl.bin_column(df.to_frame(), col='WRITE_OFF', bin_thresh = [500])
    #df1 = pd.get_dummies(df1['bin_WRITE_OFF'])
    #df1 = df1.drop(0, axis=1)
    #df1.columns = ['WRITE_OFF_LABEL']
    df1 = df.copy()
    df1['WRITE_OFF_LABEL'] = ((df1['WRITE_OFF'] > 500) | (df1['WRITE_OFF'] > 0.05 * df1['TOTAL_FEES'])).astype(int)
    df1.drop(columns=['WRITE_OFF'], inplace=True)
    return df1

In [ ]:
df_temp = pr.read_excel(raw_data_path)
print(convert_y(df_test)['WRITE_OFF_LABEL'].value_counts())

In [ ]:
df_y_train_new = convert_y(df_y_train)
df_y_test_new = convert_y(df_y_test)
print(df_y_train_new['WRITE_OFF_LABEL'].value_counts())
print(df_y_test_new['WRITE_OFF_LABEL'].value_counts())

## DECISION TREE

### Decision Tree WITHOUT SMOTE

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.tree import export_graphviz
from subprocess import call

In [ ]:
criterion = 'entropy'
rand_seed = 0

In [ ]:
dt_cls = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
dt_cls.fit(df_X_train_new, df_y_train_new)
print(dt_cls)
print(f"Accuracy on training set without SMOTE decision tree: {dt_cls.score(df_X_train_new, df_y_train_new)}")
print(f"Accuracy on test set without SMOTE decision tree: {dt_cls.score(df_X_test_new, df_y_test_new)}")

filename = file_dir + 'all_hosp_uc1_no_smote_dt_model.pkl'
pickle.dump(dt_cls, open(filename, 'wb'))

y_pred = dt_cls.predict(df_X_test_new)
y_pred.to_pickle(file_dir + 'all_hosp_uc1_no_smote_predictions.pkl')
print('no SMOTE decision tree \n', confusion_matrix(df_y_test_new, y_pred))  
print('no SMOTE decision tree \n', classification_report(df_y_test_new, y_pred)) 

### Decision Tree WITH SMOTE

#### Decision Tree WITH SMOTE SS = 0.1 ~ 1.0

In [ ]:
for sample in np.arange(0.1, 1.1, 0.1):
    sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
    X_train_ss, y_train_ss = sm_ss.fit_sample(df_X_train_new, df_y_train_new)

    model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
    model.fit(X_train_ss, y_train_ss)
    print(model)
    print(f"Accuracy on training set with {sample} SMOTE decision tree: {model.score(X_train_ss, y_train_ss)}")
    print(f"Accuracy on test set with {sample} SMOTE decision tree: {model.score(df_X_test_new, df_y_test_new)}")

    filename = file_dir + f'all_hosp_uc1_{sample}_smote_dt_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    y_pred = model.predict(df_X_test_new)
    print(f'{sample} SMOTE decision tree \n', confusion_matrix(df_y_test_new, y_pred))  
    print(f'{sample} SMOTE decision tree \n', classification_report(df_y_test_new, y_pred)) 

## LOGISTIC REGRESSION

### Logistic Regression Without SMOTE

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg_strength = 0.01

In [ ]:
logreg = LogisticRegression(C=reg_strength).fit(df_X_train_new, df_y_train_new)
print(logreg)
print(f"Training set without SMOTE log reg score: {logreg.score(df_X_train_new, df_y_train_new)}")
print(f"Test set without SMOTE log reg score: {logreg.score(df_X_test_new, df_y_test_new)}")

filename = file_dir + 'all_hosp_uc1_no_smote_logreg_model.pkl'
pickle.dump(logreg, open(filename, 'wb'))

y_pred = logreg.predict(df_X_test_new)
print('no SMOTE log reg \n', confusion_matrix(df_y_test_new, y_pred))  
print('no SMOTE log reg \n', classification_report(df_y_test_new, y_pred)) 

### Logistic Regression With SMOTE

#### Logistic Regression With SMOTE ss=0.1 ~ 1.0

In [ ]:
for sample in np.arange(0.1, 1.1, 0.1):
    sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
    X_train_ss, y_train_ss = sm_ss.fit_sample(df_X_train_new, df_y_train_new)

    model = LogisticRegression(C=reg_strength).fit(X_train_ss, y_train_ss)
    print(model)
    print(f"Training set with {sample} SMOTE log reg score: {model.score(X_train_ss, y_train_ss)}")
    print(f"Test set with {sample} SMOTE log reg score: {model.score(df_X_test_new, df_y_test_new)}")

    filename = file_dir + f'all_hosp_uc1_{sample}_smote_logreg_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    y_pred = model.predict(df_X_test_new)
    print(f'{sample} SMOTE log reg \n', confusion_matrix(df_y_test_new, y_pred))  
    print(f'{sample} SMOTE log reg \n', classification_report(df_y_test_new, y_pred)) 

## NAIVE BAYES

### Naive Bayes Without SMOTE

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Initiating the Gaussian Classifier
gaussnb = GaussianNB()

# Training your model 
gaussnb.fit(df_X_train_new, df_y_train_new)
print(gaussnb)
filename = file_dir + 'all_hosp_uc1_no_smote_nb_model.pkl'
pickle.dump(gaussnb, open(filename, 'wb'))

# Score
print(f"Training set without SMOTE Naive Bayes score: {gaussnb.score(df_X_train_new, df_y_train_new)}")
print(f"Test set without SMOTE Naive Bayes score: {gaussnb.score(df_X_test_new, df_y_test_new)}")

# Confusion Matrix
y_pred = gaussnb.predict(df_X_test_new)
print('no SMOTE Naive Bayes \n', confusion_matrix(df_y_test_new, y_pred))  
print('no SMOTE Naive Bayes \n', classification_report(df_y_test_new, y_pred)) 

### Naive Bayes With SMOTE

#### Naive Bayes Without SMOTE ss=0.1 ~ 1.0

In [ ]:
for sample in np.arange(0.1, 1.1, 0.1):
    sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
    X_train_ss, y_train_ss = sm_ss.fit_sample(df_X_train_new, df_y_train_new)

    # Initiating the Gaussian Classifier
    model = GaussianNB()

    # Training your model 
    model.fit(X_train_ss, y_train_ss)
    print(model)
    filename = file_dir + f'all_hosp_uc1_{sample}_smote_nb_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    # Score
    print(f"Training set with {sample} SMOTE Naive Bayes score: {model.score(X_train_ss, y_train_ss)}")
    print(f"Test set with {sample} SMOTE Naive Bayes score: {model.score(df_X_test_new, df_y_test_new)}")

    # Confusion Matrix
    y_pred = model.predict(df_X_test_new)
    print(f'{sample} Naive Bayes\n', confusion_matrix(df_y_test_new, y_pred))  
    print(f'{sample} Naive Bayes\n', classification_report(df_y_test_new, y_pred)) 

## NEURAL NET MLP

### Neural Net MLP Without SMOTE, 2 Layers

In [ ]:
from sklearn.neural_network import MLPClassifier  
from sklearn import metrics

In [ ]:
list_hid_layers = [(128,128), (16,16,16), (32,32,32), (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = np.arange(0.1, 1.1, 0.1)#[0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net MLPClassifier without SMOTE

In [ ]:
for hid_layers in list_hid_layers:
    model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
    model.fit(df_X_train_new, df_y_train_new)
    print(model)
    filename = file_dir + f'all_hosp_uc1_no_smote_mlp_{hid_layers}_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    predictions = mlp.predict(df_X_test_new) 

    # Score
    print(f"Training set without SMOTE mlp {hid_layers} score: {model.score(df_X_train_new, df_y_train_new)}")
    print(f"Test set without SMOTE mlp {hid_layers} score: {model.score(df_X_test_new, df_y_test_new)}")

    print(f"Accuracy without SMOTE mlp {hid_layers}: ", metrics.accuracy_score(df_y_test_new, predictions))
    print(f'no SMOTE mlp {hid_layers}\n', confusion_matrix(df_y_test_new,predictions))  
    print(f'no SMOTE mlp {hid_layers}\n', classification_report(df_y_test_new,predictions))  

### Neural Net With SMOTE

In [ ]:
list_hid_layers = [(16,16), (32,32), (64,64), (128,128), (256,256),
                   (16,16,16), (32,32,32), (64,64,64),
                   (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = np.arange(0.1, 1.1, 0.1)#[0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net Layers With SMOTE

In [ ]:
for hid_layers in list_hid_layers:
    for sample in list_smote_sampling:
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(df_X_train_new, df_y_train_new)

        model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
        model.fit(X_train_ss, y_train_ss)  
        print(model)
        filename = file_dir + f'all_hosp_uc1_{sample}_smote_mlp_{hid_layers}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        predictions = model.predict(df_X_test_new) 

        # Score
        print(f"Training set {sample} SMOTE mlp {hid_layers} score: {model.score(X_train_ss, y_train_ss)}")
        print(f"Test set {sample} SMOTE mlp {hid_layers} score: {model.score(df_X_test_new, df_y_test_new)}")

        print(f"Accuracy: ", metrics.accuracy_score(df_y_test_new, predictions))
        print(f'{sample} SMOTE mlp {hid_layers}\n', confusion_matrix(df_y_test_new,predictions))  
        print(f'{sample} SMOTE mlp {hid_layers}\n', classification_report(df_y_test_new,predictions))  

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# From the above, the best KNN is at k=8
#Setup a knn classifier with k neighbors
best_knn = KNeighborsClassifier(8)

### KNN without SMOTE

In [ ]:
# Fit the KNN model with training data and score the test data
best_knn = KNeighborsClassifier(n_neighbours)

best_knn.fit(df_X_train_new,df_y_train_new)
best_knn.score(df_X_test_new,df_y_test_new)

# save the model to disk
filename = f'all_hosp_knn_without_smote.pkl'
pickle.dump(best_knn, open(filename, 'wb'))

y_pred = best_knn.predict(df_X_test_new)
# Score for Best KNN, No SMOTE
print(f"Training set without SMOTE KNN({n_neighbours}) score: {best_knn.score(df_X_train_new, df_y_train_new)}")
print(f"Test set without SMOTE KNN({n_neighbours}) score: {best_knn.score(df_X_test_new, df_y_test_new)}")

print(f"Accuracy without SMOTE KNN({n_neighbours}): {metrics.accuracy_score(df_y_test_new, y_pred)}")
print(f'no SMOTE KNN({n_neighbours})\n', confusion_matrix(df_y_test_new,y_pred))  
print(f'no SMOTE KNN({n_neighbours})\n', classification_report(df_y_test_new,y_pred))  

### KNN With SMOTE

#### KNN With SMOTE ss=0.1

In [ ]:
list_smote_sampling = np.arange(0.1, 1.1, 0.1)#[0.1, 0.5, 1.0]
for sample in list_smote_sampling:
    sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
    X_train_ss, y_train_ss = sm_ss.fit_sample(df_X_train_new, df_y_train_new)

    # Fit the KNN model with training data and score the test data
    model = KNeighborsClassifier(n_neighbours)    
    model.fit(X_train_ss,y_train_ss)
    #model.score(df_X_test_new,df_y_test_new)

    # save the model to disk
    filename = f'all_hosp_knn{n_neighbours}_{sample}_smote.pkl'
    pickle.dump(model, open(filename, 'wb'))

    y_pred = model.predict(df_X_test_new)
    # Score for Best KNN, with SMOTE
    print(f"Training set {sample} SMOTE KNN({n_neighbours}) score: {model.score(X_train_ss, y_train_ss)}")
    print(f"Test set {sample} SMOTE KNN({n_neighbours}) score: {model.score(df_X_test_new, df_y_test_new)}")

    print(f"Accuracy {sample} SMOTE KNN({n_neighbours}): ", metrics.accuracy_score(df_y_test_new, y_pred))
    print(f'{sample} SMOTE KNN({n_neighbours})\n', confusion_matrix(df_y_test_new,y_pred))  
    print(f'{sample} SMOTE KNN({n_neighbours})\n', classification_report(df_y_test_new,y_pred)) 